In [68]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import numpy as np
%matplotlib inline
import re 
import math
import time

In [25]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
import spacy
# Need to load the large model to get the vectors
import en_core_web_sm
nlp = en_core_web_sm.load()

In [27]:
import os
import pandas as pd

# read the text file and add the column names
read_file = pd.read_csv(r"booksummaries.txt", sep='	', header=None)
read_file.columns = ['ID', 'm number', 'book name', 'author name', 'date', 'label', 'summary']

# clean data
read_file['label'] = read_file['label'].str.replace(r'/m/\S*\s', '')
read_file['label'] = read_file['label'].str.replace(r'{', '')
read_file['label'] = read_file['label'].str.replace(r'}', '')
read_file['label'] = read_file['label'].str.replace(r'}', '')
read_file['label'] = read_file['label'].str.replace(r'\\u00e0\s+clef', '')

# select columns
new_file = read_file.loc[:, ['book name', 'label', 'summary']]

#delete the columns with no labels
new_file.dropna(axis = 0, how = 'any', inplace = True)
new_file = new_file.iloc[:, [0, 2, 1]]

new_file = new_file.reset_index(drop=True)

#output data as csv
new_file.to_csv(r'./booksummries.csv', index=False)


In [28]:
new_file.head()

,book name,summary,label
0,Animal Farm,"Old Major, the old boar on the Manor Farm, ca...","""""Roman "", """"Satire"", """"Children's literature""..."
1,A Clockwork Orange,"Alex, a teenager living in near-future Englan...","""""Science Fiction"", """"Novella"", """"Speculative ..."
2,The Plague,The text of The Plague is divided into five p...,"""""Existentialism"", """"Fiction"", """"Absurdist fic..."
3,A Fire Upon the Deep,The novel posits that space around the Milky ...,"""""Hard science fiction"", """"Science Fiction"", ""..."
4,All Quiet on the Western Front,"The book tells the story of Paul Bäumer, a Ge...","""""War novel"", """"Roman """


In [29]:
def text_process(label_list):
    has_fiction = False
    has_spec_fiction = False
    has_novel = False
    has_spec_novel = False
    for i in range(len(label_list)):
        if 'novel' in label_list[i].lower():
            if 'novel' == label_list[i].lower():
                has_novel = True
            else:
                has_spec_novel = True
        if 'fiction' in label_list[i].lower():
            if 'fiction' == label_list[i].lower():
                has_fiction = True
            else:
                has_spec_fiction = True
        
    if has_spec_fiction and has_spec_novel:
        if has_fiction:
            label_list.remove('fiction')
        if has_novel:
            label_list.remove('novel')
    elif has_spec_fiction:
        if has_fiction:
            label_list.remove('fiction')
        if has_novel:
            label_list.remove('novel')
    elif has_spec_novel:
        if has_fiction:
            label_list.remove('fiction')
        if has_novel:
            label_list.remove('novel')
    elif has_fiction and has_novel:
        label_list.remove('fiction')
    return label_list

In [30]:
for index in range(len(new_file['label'])):
    label = new_file['label'][index].replace('"', ''). lower()
    label_list = re.split(', ', label)  
    label_list = text_process(label_list)
    new_file.xs(index)['label']= label_list

In [31]:
#output data as csv
new_file.to_csv(r'./booksummries.csv', index=False)


In [32]:
new_file.tail()

,book name,summary,label
12836,The Third Lynx,The story starts with former government agent...,[science fiction]
12837,Remote Control,The series follows the character of Nick Ston...,"[thriller, fiction, suspense]"
12838,Transfer of Power,The reader first meets Rapp while he is doing...,"[thriller, fiction]"
12839,Decoded,The book follows very rough chronological ord...,[autobiography]
12840,Poor Folk,Makar Devushkin and Varvara Dobroselova are s...,"[epistolary novel, speculative fiction]"


  ## Encoding the Labels##




In [33]:
label_list = []
for index in range(len(new_file['label'])):
    object_label = new_file['label'][index]
    for l in object_label:
        if l not in label_list:
            label_list.append(l)

In [34]:
len(label_list)

227

In [35]:
one_hot = pd.DataFrame(np.zeros((12841, 227)), columns=label_list).astype(int)

In [36]:
for index in range(len(new_file['label'])):
    object_label = new_file['label'][index]
    for l in object_label:
        one_hot[l][index] = 1

In [37]:
one_hot

,roman,satire,children's literature,speculative fiction,science fiction,novella,utopian and dystopian fiction,existentialism,absurdist fiction,hard science fiction,...,encyclopedia,mashup,biopunk,popular culture,neuroscience,new york times best seller list,epic science fiction and fantasy,alien invasion,prose,pastiche
0,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12836,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12837,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12838,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12839,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Split the words##

In [38]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


def words_process(new_file):
    book_summaries = new_file['summary']
    summary_list = [summary for summary in book_summaries]
    summary_num = len(summary_list)
    #summaries = ''.join(summary_list)
    print("the total number of books: {}\n".format(summary_num))
    
    all_docs = []

    for doc in summary_list:
        # Tokenize the string into words
        tokens = word_tokenize(doc)
        # Remove non-alphabetic tokens, such as punctuation
        words = [word.lower() for word in tokens if word.isalpha()]
        stop_words = set(stopwords.words('english'))
        words = [word for word in words if not word in stop_words]
        all_docs.append(words)
        
    return all_docs

In [39]:
all_words = words_process(new_file.iloc[:, :]) ####NEED TO CHANGE TO ALL!!!!!!!

the total number of books: 12841



## outliners ##

## Using a Pre-Trained Embedding Layer

In [ ]:
# import Word2Vec loading capabilities
from gensim.models import KeyedVectors

# Creating the model
embed_lookup = KeyedVectors.load_word2vec_format('word2vec_model/word2vec-slim/GoogleNews-vectors-negative300.bin', 
                                                 binary=True)


In [ ]:
# store pretrained vocab
pretrained_words = []
for word in embed_lookup.vocab:
    pretrained_words.append(word)

In [58]:
row_idx = 1

# get word/embedding in that row
word = pretrained_words[row_idx] # get words by index
embedding = embed_lookup[word] # embeddings by word

# vocab and embedding info
print("Size of Vocab: {}\n".format(len(pretrained_words)))
print('Word in vocab: {}\n'.format(word))
print('Length of embedding: {}\n'.format(len(embedding)))
#print('Associated embedding: \n', embedding)

Size of Vocab: 3000000

Word in vocab: in

Length of embedding: 300



# Tokenize reviews

In [20]:
def tokenize_all_sum(embed_lookup):
    tokenized_sum = []
    for summ in all_words:
        ints = []
        for w in summ:
            try:
                idx = embed_lookup.vocab[w].index
            except: 
                idx = 0
            ints.append(idx)
        tokenized_sum.append(ints)
    return tokenized_sum

In [21]:
tokenized_sum = tokenize_all_sum(embed_lookup)

In [22]:
# testing code and printing a tokenized review
print(tokenized_sum[0])

[154, 338, 154, 86316, 63917, 2563, 926, 2418, 2563, 349, 8691, 5103, 32366, 7328, 2418, 8709, 2216, 146659, 338, 6119, 54, 533, 13250, 37391, 517953, 4315, 3929, 749, 2768, 5868, 2418, 14076, 817, 18500, 11132, 94620, 131280, 2563, 35097, 2801, 2563, 4922, 375, 60262, 396, 2418, 3325, 37391, 2687, 4016, 2418, 1772, 2009, 560, 18855, 2563, 623, 9841, 13250, 18603, 1784, 1274, 211, 3125, 605, 560, 1281, 22341, 764, 492, 517953, 920, 38308, 2563, 2766, 5325, 9223, 517953, 37391, 2894, 1274, 37391, 5513, 383, 857, 45282, 517953, 2766, 5148, 37391, 309, 15156, 614, 517953, 75114, 726, 6481, 2351, 2563, 5226, 1778, 956, 13250, 209, 2563, 527, 533, 13283, 945, 798371, 34858, 517953, 1059, 912, 45282, 931, 2418, 141, 3437, 3102, 2200, 870, 45282, 3148, 1896, 2418, 359, 45282, 66096, 517953, 798371, 5823, 2418, 37391, 3016, 1021, 34168, 0, 2470, 3064, 4406, 5160, 37391, 2540, 29160, 517953, 2219, 50487, 2563, 2766, 1423, 2418, 11092, 174391, 154, 2318, 13250, 2258, 325, 9174, 509, 36632, 12891

In [40]:
max_Length = 0  
for x in tokenized_sum:
    if len(x) > max_Length:
        max_Length = len(x)
max_Length

2975

## Padding sequences

In [41]:
def pad_features(tokenized_sum, seq_length):
    
    # getting the correct rows x cols shape
    features = np.zeros((len(tokenized_sum), seq_length), dtype=int)
 
    # for each review, I grab that review and 
    for i, row in enumerate(tokenized_sum):
        if len(row)>0:
            features[i, -len(row):] =  np.array(row)[:seq_length]
    
    return features

In [42]:
features = pad_features(tokenized_sum, max_Length)

assert len(features)==len(tokenized_sum), "Features should have as many rows as reviews."
assert len(features[0])==max_Length, "Each feature row should contain seq_length values."

print(features[2])

[    0     0     0 ... 13421 40364  5103]


# Training, Validation, and Test Data

In [43]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = one_hot[:split_idx], one_hot[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))


			Feature Shapes:
Train set: 		(10272, 2975) 
Validation set: 	(1284, 2975) 
Test set: 		(1285, 2975)


### Above only 2,0,1 because I only used  3 datasample to do all of these

# DataLoaders and Batching

In [169]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(np.array(train_x)), torch.from_numpy(np.array(train_y)[:,0]))
valid_data = TensorDataset(torch.from_numpy(np.array(val_x)), torch.from_numpy(np.array(val_y)))
test_data = TensorDataset(torch.from_numpy(np.array(test_x)), torch.from_numpy(np.array(test_y)))

# dataloaders
batch_size = 50
RANDOM_SEED = 1
NUM_EPOCHS = 10
NUM_CLASSES = 227
GRAYSCALE = False
LEARNING_RATE = 0.001

# shuffling and batching data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [138]:
train_x.shape

(10272, 2975)

In [168]:
np.array(train_y)[:,0].shape

(10272,)

In [170]:
#First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

No GPU available, training on CPU.


In [171]:
import torch.nn as nn
import torch.nn.functional as F

class SentimentCNN(nn.Module):
    """
    The embedding layer + CNN model that will be used to perform sentiment analysis.
    """

    def __init__(self, embed_model, vocab_size, output_size, embedding_dim,
                 num_filters=100, kernel_sizes=[3, 4, 5], freeze_embeddings=True, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentCNN, self).__init__()

        # set class vars
        self.num_filters = num_filters
        self.embedding_dim = embedding_dim
        
        # 1. embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # set weights to pre-trained
        self.embedding.weight = nn.Parameter(torch.from_numpy(embed_model.vectors)) # all vectors
        # (optional) freeze embedding weights
        if freeze_embeddings:
            self.embedding.requires_grad = False
        
        # 2. convolutional layers
        self.convs_1d = nn.ModuleList([
            nn.Conv2d(1, num_filters, (k, embedding_dim), padding=(k-2,0)) 
            for k in kernel_sizes])
        
        # 3. final, fully-connected layer for classification
        self.fc = nn.Linear(len(kernel_sizes) * num_filters, output_size) 
        
        # 4. dropout and sigmoid layers
        self.dropout = nn.Dropout(drop_prob)
        self.sig = nn.Sigmoid()
        
    
    def conv_and_pool(self, x, conv):
        """
        Convolutional + max pooling layer
        """
        # squeeze last dim to get size: (batch_size, num_filters, conv_seq_length)
        # conv_seq_length will be ~ 200
        x = F.relu(conv(x)).squeeze(3)
        
        # 1D pool over conv_seq_length
        # squeeze to get size: (batch_size, num_filters)
        x_max = F.max_pool1d(x, x.size(2)).squeeze(2)
        return x_max

    def forward(self, x):
        """
        Defines how a batch of inputs, x, passes through the model layers.
        Returns a single, sigmoid-activated class score as output.
        """
        # embedded vectors
        embeds = self.embedding(x) # (batch_size, seq_length, embedding_dim)
        # embeds.unsqueeze(1) creates a channel dimension that conv layers expect
        embeds = embeds.unsqueeze(1)
        
        # get output of each conv-pool layer
        conv_results = [self.conv_and_pool(embeds, conv) for conv in self.convs_1d]
        
        # concatenate results and add dropout
        x = torch.cat(conv_results, 1)
        x = self.dropout(x)
        
        # final logit
        logit = self.fc(x) 
        probas = F.softmax(logit, dim=1)
       
        # sigmoid-activated --> a class score
        return self.sig(logit)


# Instantiate the network

In [ ]:
# Instantiate the model w/ hyperparams

vocab_size = len(pretrained_words)
output_size = 1 # binary class (1 or 0)
embedding_dim = len(embed_lookup[pretrained_words[0]]) # 300-dim vectors
num_filters = 100
kernel_sizes = [3, 4, 5]

net = SentimentCNN(embed_lookup, vocab_size, output_size, embedding_dim,
                   num_filters, kernel_sizes)

print(net)

# Train

In [176]:
# loss and optimization functions
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [177]:
# training loop
def train(net, train_loader, epochs, print_every=100):

    # move model to GPU, if available
    if(train_on_gpu):
        net.cuda()

    counter = 0 # for printing
    
    # train for some number of epochs
    net.train()
    for e in range(epochs):

        # batch loop
        for inputs, labels in train_loader:
            counter += 1

            if(train_on_gpu):
                inputs, labels = inputs.cuda(), labels.cuda()

            # zero accumulated gradients
            net.zero_grad()

            # get the output from the model
            logit = net(inputs)
            
            # calculate the loss and perform backprop
            loss = criterion(logit.squeeze(), labels.float())
            loss.backward()
            optimizer.step()

            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_losses = []
                net.eval()
                for inputs, labels in valid_loader:

                    if(train_on_gpu):
                        inputs, labels = inputs.cuda(), labels.cuda()

                    output = net(inputs)
                    val_loss = criterion(output.squeeze(), labels.float())

                    val_losses.append(val_loss.item())

                net.train()
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.6f}...".format(loss.item()),
                      "Val Loss: {:.6f}".format(np.mean(val_losses)))

In [ ]:
# training params

epochs = 2 # this is approx where I noticed the validation loss stop decreasing
print_every = 100

train(net, train_loader, epochs, print_every=print_every)